<h1 style="color:green">NBU-powered Scratchpad</h1>
<p style="color:green">This notebook is used for code smithening purposes</p>
<p style="color:green">It includes notebook utilities support, which means it can easily invoke functionality from the Conway-based project it belongs to</p>
<p>NB: this notebook requires to be launched in an environment that includes conway and conway.ops</p>

In [1]:
import os                                                               as _os
import sys
sys.path.append(_os.path.dirname(_os.path.dirname(_os.getcwd())) + "/src") # So we can import scratch_nb_apps
from scratch_nb_apps.scratch_nb_utils import Scratch_NB_Utils
NBU                       = Scratch_NB_Utils()
DFU                       = NBU.DFU
T0                        = NBU.time.perf_counter()

SCRATCH installation:                scratch_fork    
Jupyter using repo[branch]:      scratch.ops[afin-dev]    
Installation path:               /home/alex/consultant1@CCL/dev/scratch_fork    
Application:                     <class 'scratch_nb_apps.scratch_nb_application.Scratch_NB_Application'>    


<h1 style="color:purple">Graph of Awaitables</h1>
<p> We pilot whether we can maintain a graph of awaitables, including:</p>
<li> Multiple dependencies: awaitables B, C both awaiting on a common awaitable A

In [2]:
import asyncio

In [18]:
async def A() -> str:
    print("A starts ...", flush=True)
    await asyncio.sleep(1)
    print("... A ends", flush=True)
    return "A"

In [22]:
async def B(an_a) -> str:
    print("B starts ...", flush=True)
    x = await an_a #A()
    print("... B ends", flush=True)
    return "B"

In [23]:
async def supervisor():
    result_l                                        =  []
    print("Scheduling A ...", flush=True)
    a = A()
    print("... A scheduled", flush=True)
    print("Scheduling B ...", flush=True)
    b = B(a)
    print("... B scheduled", flush=True)
    to_do = [a, b]
    to_do_iter                                      = asyncio.as_completed(to_do)
    for coro in to_do_iter:
        coro_result                                 = await coro
        result_l.append(coro_result)
    return result_l

In [24]:
#result_l = asyncio.run(supervisor()) # Don't do this in Jupyter since Jupyter is already running an event loog
result_l = await supervisor()
result_l

Scheduling A ...
... A scheduled
Scheduling B ...
... B scheduled
B starts ...
A starts ...
... A ends
... B ends


RuntimeError: cannot reuse already awaited coroutine